All import statements and variable initializations have been done  

In [1]:
from sequence_lableler_final import Sequence_labeler , parse_config
import os
import sys

In [2]:
path = "conf/fcepublic.conf"
temp_model_path = "model/model"
config = parse_config("config", path)
file_paths = { "train": config["path_train"] , "dev" : config["path_dev"], "test" : config["path_test"]}

In [3]:
labeler = Sequence_labeler(config)

In [4]:
dataset = labeler.read_dataset(file_paths,"ConLL")

inside read_dataset
train 91
dev 30
test 141


read and load the datasets for train , test and dev

In [5]:
dataset_formatted = dict()
dataset_formatted["train"] = labeler.data_formatted(dataset["train"])
dataset_formatted["dev"] = labeler.data_formatted(dataset["dev"])
dataset_formatted["test"] = labeler.data_formatted(dataset["test"])
data_train, data_dev, data_test = dataset_formatted["train"],dataset_formatted["dev"],dataset_formatted["test"]

building the vocabulary 

In [6]:
labeler.build_vocabs(data_train, data_dev, data_test, config["preload_vectors"])
labeler.construct_network()
labeler.initialize_session()



 in build vocab  
Counter({'O': 187, 'I-WORK_OF_ART': 17, 'B-PERSON': 9, 'B-WORK_OF_ART': 6, 'I-FAC': 3, 'I-PERSON': 3, 'B-ORDINAL': 2, 'TRUE_LABEL': 1, 'B-GPE': 1, 'B-CARDINAL': 1, 'B-FAC': 1})  label_counter
n_words: 88
n_chars: 49
n_labels: 11
n_singletons: 17


/Users/aishwaryaadiga/anaconda2/envs/python3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


loading word embeddings and models preloaded

In [7]:
if config["preload_vectors"] != None:
    labeler.preload_word_embeddings(config["preload_vectors"])
if config["load"] != None and len(config["load"])  > 0 and os.path.exists(config["load"]):
    try:
        labeler.load_model(labeler.session, temp_model_path)
    except:
        print("error in loading the model")

n_preloaded_embeddings: 85


Training the model 

In [8]:
labeler.train(data_train,data_dev,temp_model_path)

EPOCH: 0
current_learningrate: 1.0
precision:  0.7727272727272727  recall:  0.9444444444444444  F1 :  0.85
saving  model/model
inside save
model_pickle.p
best_epoch: 0
EPOCH: 1
current_learningrate: 1.0
precision:  0.75  recall:  1.0  F1 :  0.8571428571428571
saving  model/model
inside save
model_pickle.p
best_epoch: 1
EPOCH: 2
current_learningrate: 1.0
precision:  0.75  recall:  1.0  F1 :  0.8571428571428571
best_epoch: 1
EPOCH: 3
current_learningrate: 1.0
precision:  0.75  recall:  1.0  F1 :  0.8571428571428571
best_epoch: 1


predictions 

In [10]:
predictions_formatted = labeler.predict(data_test)

predictions are printed in the format [start index, span, token, token type] - 
The first two indices are None in this case for all predictions 


In [13]:
print(predictions_formatted[2])

[[None, None, 'but', 'O'], [None, None, 'I', 'O'], [None, None, 'am', 'O'], [None, None, 'not', 'O'], [None, None, 'selling', 'O'], [None, None, 'medicine', 'O'], [None, None, 'or', 'O'], [None, None, 'pharmaceuticals', 'O'], [None, None, '/.', 'O']]


In [14]:
cost = 0
predicted_labels = []
groundTruths = []

for i in range(len(predictions_formatted)):
    predicted_labels_sent = []
    groundTruths_sent = []
    for j in range(len(predictions_formatted[i])):
        predicted_labels_sent.append(labeler.label2id[predictions_formatted[i][j][-1]])

    predicted_labels.append(predicted_labels_sent)


for i in data_test:
    groundTruths_sent = []
    for item in i : 
        groundTruths_sent.append(item)
    groundTruths.append(groundTruths_sent)

In [15]:
with open('result.txt','w',encoding = 'utf-8') as f:
    for i in range(len(data_test)):
        for j in range(len(data_test[i])):


            f.write(str(data_test[i][j][0])+" "+str(data_test[i][j][3])+ " " +  str(labeler.id2label[predicted_labels[i][j]]))
            f.write("\n")

        f.write("\n")

In [16]:
precision,recall,f1 = labeler.evaluate(predicted_labels,groundTruths, cost,"test")

This result is on a  small sample dataset, actual numbers are reported in the presentation 

In [17]:
print(precision,recall,f1)

0.765625 1.0 0.8672566371681416
